# RAG 챗봇 구현

In [5]:
# 라이브러리 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

## 인덱싱 과정
### 문서 준비 및 분할
- 문서를 청크로 분할하여 벡터 데이터베이스에 저장합니다. 청크는 의미 단위로 나누되, 크기가 너무 작거나 크지 않도록 적절히 설정해 검색 성능과 정보 보존을 최적화해야 합니다.

In [2]:
# PDF 문서 로드 및 텍스트 분할
loader = PyPDFLoader("2024_KB_부동산_보고서_최종.pdf")
documents = loader.load() # PDF 문서 로드
#텍스트 분할 설정: 청크 크기와 겹침 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents) # 문서 청크 분할

#분할 된 청크 수 확인
print(f"분할된 청크의 수: {len(chunks)}") # 총 청크 수 출력

분할된 청크의 수: 135


### 임베딩 생성과 데이터베이스 적재, 관리
- 청크로 분할된 문서를 임베딩 모델을 통해 벡터화하여 벡터 데이터베이스에 저장합니다.

In [20]:
# 임베딩 생성 및 Chroma 데이터베이스에 저장
embedding_function = OllamaEmbeddings(model="bge-m3")
persist_directory = "chroma_db" # Chroma 데이터베이스 디렉토리
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory
)
print("문서의 수:", vectorstore._collection.count()) # Chroma 데이터베이스에 저장된 문서 수 출력

문서의 수: 405


## 쿼리 과정
### 검색 및 재정렬
- 사용자가 입력한 텍스트를 벡터로 변환한 후, 데이터베이스 검색에 활용할 수 있도록 처리합니다.
- 리트리버는 챗봇 실행 시 내부적으로 호출되어 동작하는 검색 도구로, 사용자의 텍스트 입력을 기반으로 검색 기능을 수행합니다.
- as_retriever() 메서드는 벡터 저장소에서 문서를 검색하는 검색기를 생성하는 데 사용합니다.

In [12]:
# 리트리버 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # k=3으로 설정된 리트리버 생성

### 프롬프트 템플릿 설정
- 검색된 청크들을 조합하여 질문에 적합한 프롬프트를 구성합니다.

In [6]:
# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: {context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0) # AI 모델 설정

- system에는 모델의 전반적인 역할과 행동 지침을 설정합니다.
- placeholder는 대화 기록용으로 이전 대화 내용을 삽입할 수 있습니다.
- human은 사용자 질문이나 입력을 나타냅니다.

In [7]:
print(prompt.format(context="컨텍스트 예시", chat_history=["대화 기록 예시1","대화 기록 예시2"], question="질문 예시")) # 프롬프트 템플릿 포맷팅 예시

System: 당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: 컨텍스트 예시

Human: 대화 기록 예시1
Human: 대화 기록 예시2
Human: 질문 예시


### 문맥 구성 및 챗봇 체인 구성
- 검색한 문서 내용을 format_docs로 정리하여 프롬프트에 전달하고, 모델이 이를 바탕으로 답변을 생성합니다.
- 검색된 문서 청크를 하나의 텍스트로 조합하고 이를 프롬프트에 전달하여 답변을 생성합니다.
- format_docs는 문서 청크를 연결하고, RunnablePassthrough는 이를 프롬프트에 전달하며, StrOutputParser는 모델 출력을 문자열로 반환합니다.

In [8]:
# 문서 형식 변환 함수 정의
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) #문서를 줄바꿈으로 연결
#체인 구성: 검색한 문서를 프롬프트에 연결하고 모델을 통해 응답 생성
chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x["question"]))
    ) # 검색된 문서를 연결하여 전달
    | prompt
    | model
    | StrOutputParser() # 결과를 문자열로 변환
)

### 메모리 설정 및 챗봇 실행
- 챗봇을 구현할 때, 대화 기록을 유지하고 세션별로 관리하여 사용자와 연속적인 대화를 지원하도록 설정할 수 있습니다.
- ChatMessageHistory 객체는 세션별 대화 기록을 유지하고, 이를 통해 챗봇이 대화의 연속성을 갖출 수 있게 합니다.
- RunnableWithMessageHistory는 대화 기록을 통해 이전 메시지를 history_messages_key로 전달하여, 사용자가 이전에 했던 질문의 맥락을 챗봇이 기억하도록 돕습니다.

In [9]:
# 대화 기록을 유지하기 위한 메모리 설정
chat_history = ChatMessageHistory() # 대화 기록 초기화
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,    #세션 ID별 대화 기록 생성
    input_messages_key="question",
    history_messages_key="chat_history",
)

- session_id는 대화 세션을 식별하는데 사용되며, 사용자가 입력하는 질문을 chain_with_memory에 전달하여 이전 대화 내역을 바탕으로 답변을 생성하게 합니다.

In [16]:
# 챗봇 실행 함수 정의
def chat_with_bot():
    session_id = "user_session"
    print("KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")
    while True:
        user_input = input("사용자: ")
        print("사용자: ", user_input) # 사용자 입력 출력
        if user_input.lower() == "quit":
            print("챗봇을 종료합니다.")
            break

        # 사용자의 질문에 따라 chain_with_memory를 통해 응답 생성
        response = chain_with_memory.invoke(
            {"question": user_input},
            {"configurable": {"session_id": session_id}}
        )

        print("챗봇:", response) # 챗봇의 응답 출력

In [17]:
chat_with_bot() # 챗봇 실행

KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)
사용자:  랭체인 챗봇 만들기에 대해 실습중입니다.
챗봇: 랭체인 챗봇 만들기에 대해 실습 중이시군요! 챗봇을 만들기 위해 어떤 부분에서 도움이 필요하신가요? 예를 들어, 환경 설정, 데이터 준비, 모델 선택, 또는 특정 기능 구현에 대한 질문이 있으시면 말씀해 주세요. 필요한 정보를 제공해 드리겠습니다!
사용자:  방금 제가 뭐라고 이야기했나요?
챗봇: 당신은 "랭체인 챗봇 만들기에 대해 실습중입니다."라고 말씀하셨습니다. 이 내용은 랭체인(LangChain) 프레임워크를 사용하여 챗봇을 만드는 과정에 대한 언급이었습니다. 추가로 궁금한 점이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
사용자:  수도권 주택 매매 전망
챗봇: 2024년 수도권 주택 매매 전망에 대한 내용은 다음과 같습니다:

1. **하락세 지속**: 2024년에도 수도권 주택 매매가격은 하락세가 이어질 것으로 전망되고 있습니다. 부동산 시장 전문가와 공인중개사, 자산관리전문가(PB)들 중 상당수가 2024년 주택 매매가격이 하락할 것이라고 예상하고 있습니다.

2. **하락폭**: 전문가와 공인중개사들은 1~3% 하락할 것이라고 보며, PB는 3~5% 하락을 예상하는 비율이 높습니다.

3. **긍정적 기대감**: 비록 하락세가 지속될 것으로 보이지만, 시장 여건 개선에 대한 기대감이 반영되어 전문가와 공인중개사의 약 1/3이 수도권 주택 매매가격 상승을 전망하고 있습니다.

4. **규제 완화와 호재**: 정부의 규제 완화와 재건축 규제 완화, GTX 개통 등 지역별 호재가 존재하지만, 높은 금리와 DSR 규제 등으로 매수자들의 구매 여력은 여전히 제한적입니다.

5. **지역별 차이**: 서울의 주요 지역에서는 긍정적인 기대감이 유지되고 있으나, 비수도권에 대해서는 전문가들이 더 부정적인 전망을 하고 있습니다.

이와 같은 요소들이 수도권 주택 시장에 영향을 미칠 것으로 보입니다. 추가적인 질문이나 특

## 챗봇에 스트림릿 UI 적용
- streamlit은 챗봇에 손쉽게 UI를 적용할수 있는 파이썬 라이브러리 입니다.

### Streamlit
- 스트림릿을 사용하면 웹 대시보드를 쉽게 만들 수 있습니다. HTML이나 CSS 같은 웹 개발 지식이 없어도 순수 파이썬 코드만으로 데이터 분석 결과를 웹에서 즉시 시각화할 수 있습니다.

### Pyngrok
- 로컬에서 실행 중인 서버를 외부 인터넷에서도 접근할 수 있도록 해주는 터널링 도구입니다.
- 내부적으로 http://localhost:8501 과 같은 로컬 주소를 https://[무작위주소].ngrok-free.app으로 변환하여 외부에서도 접근할 수 있도록 합니다.

### ngrok 인증키 발급
- ngrok를 사용하려면 인증키를 발급 받아야 합니다.
- ngrok회원가입
- 대시보드에서 인증키 확인

- 로컬 환경에서 개발한 웹 애플리케이션을 쉽게 공유할 수 있습니다.

https://dashboard.ngrok.com/get-started/your-authtoken
```shell
ngrok config add-authtoken <본인 인증키>
```

[app.py](app.py)

- @st.cache_resource 코드는 특정 연산의 결과를 캐싱하여 실행 속도를 최적화하는 역할을 합니다.
- 동일한 데이터나 모델을 여러 번 불러올 때 불필요한 연산을 줄이고 성능을 향상시킬 수 있습니다.

1. process_pdf 함수로 PDF 문서를 로드하고 텍스트를 분할합니다.
2. initialize_vectorstore 함수에서는 임베딩 모델을 사용하여 PDF 내용을 크로마 데이터베이스에 저장합니다.
3. initialize_chain 함수는 크로마 벡터 데이터베이스를 검색기로 활용하고, 적절한 프롬프트 엔지니어링을 통해 모델 체인을 생성합니다.
4. main함수에서는 스트림릿UI를 구성하고 사용자와의 대화 내용을 관리하며, RAG 챗봇을 통해 답변을 생성합니다.

- set_page_config 함수를 사용하여 웹 페이지의 제목과 아이콘을 설정합니다.
- st.title, st.caption으로 페이지의 제목과 부제목을 표시합니다.

- session_state를 사용하여 대화 기록을 저장합니다. 페이지가 새로고침되어도 대화 내용이 유지됩니다.
- st.chat_message를 사용하여 각 메시지를 역할에 따라 구분하여 표시합니다.
- st.chat_input을 사용하여 사용자로부터 입력을 받고 st.spinner를 사용하여 응답 생성 중임을 표시하며, 생성된 응답을 st.markdown으로 표시합니다.
- st.session_state.append코드를 통해 생성된 응답을 세션 상태에 저장하며 대화 기록을 유지합니다.

In [18]:
# 터널링 및 실행
from pyngrok import ngrok

public_url = ngrok.connect(8501) # streamlit 기본 포트
print("앱 접속 URL:", public_url) # ngrok을 통해 생성된 공개 URL 출력

!streamlit run app.py

앱 접속 URL: NgrokTunnel: "https://f39c-220-70-7-1.ngrok-free.app" -> "http://localhost:8501"
^C
